# Introduction to Deep Learning with PyTorch

In this notebook, you'll get introduced to [PyTorch](http://pytorch.org/), a framework for building and training neural networks. PyTorch in a lot of ways behaves like the arrays you love from Numpy. These Numpy arrays, after all, are just tensors. PyTorch takes these tensors and makes it simple to move them to GPUs for the faster processing needed when training neural networks. It also provides a module that automatically calculates gradients (for backpropagation!) and another module specifically for building neural networks. All together, PyTorch ends up being more coherent with Python and the Numpy/Scipy stack compared to TensorFlow and other frameworks.



## Neural Networks

Deep Learning is based on artificial neural networks which have been around in some form since the late 1950s. The networks are built from individual parts approximating neurons, typically called units or simply "neurons." Each unit has some number of weighted inputs. These weighted inputs are summed together (a linear combination) then passed through an activation function to get the unit's output.

<img src="assets/simple_neuron.png" width=400px>

Mathematically this looks like: 

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i \right)
\end{align}
$$

With vectors this is the dot/inner product of two vectors:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

### Stack them up!

We can assemble these unit neurons into layers and stacks, into a network of neurons. The output of one layer of neurons becomes the input for the next layer. With multiple input units and output units, we now need to express the weights as a matrix.

<img src='assets/multilayer_diagram_weights.png' width=450px>

We can express this mathematically with matrices again and use matrix multiplication to get linear combinations for each unit in one operation. For example, the hidden layer ($h_1$ and $h_2$ here) can be calculated 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

The output for this small network is found by treating the hidden layer as inputs for the output unit. The network output is expressed simply

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

## Tensors

It turns out neural network computations are just a bunch of linear algebra operations on *tensors*, a generalization of matrices. A vector is a 1-dimensional tensor, a matrix is a 2-dimensional tensor, an array with three indices is a 3-dimensional tensor (RGB color images for example). The fundamental data structure for neural networks are tensors and PyTorch (as well as pretty much every other deep learning framework) is built around tensors.

<img src="assets/tensor_examples.svg" width=600px>

With the basics covered, it's time to explore how we can use PyTorch to build a simple neural network.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch

import helper

First, let's see how we work with PyTorch tensors. These are the fundamental data structures of neural networks and PyTorch, so it's imporatant to understand how these work.

In [2]:
x = torch.rand(3, 2)
x

tensor([[0.5974, 0.9384],
        [0.4729, 0.3444],
        [0.9928, 0.7016]])

In [4]:
y = torch.ones(x.size())
y

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [5]:
z = x + y
z

tensor([[1.5974, 1.9384],
        [1.4729, 1.3444],
        [1.9928, 1.7016]])

In general PyTorch tensors behave similar to Numpy arrays. They are zero indexed and support slicing.

In [6]:
z[0]

tensor([1.5974, 1.9384])

In [7]:
z[:, 1:]

tensor([[1.9384],
        [1.3444],
        [1.7016]])

Tensors typically have two forms of methods, one method that returns another tensor and another method that performs the operation in place. That is, the values in memory for that tensor are changed without creating a new tensor. In-place functions are always followed by an underscore, for example `z.add()` and `z.add_()`.

In [8]:
# Return a new tensor z + 1
z.add(1)

tensor([[2.5974, 2.9384],
        [2.4729, 2.3444],
        [2.9928, 2.7016]])

In [9]:
# z tensor is unchanged
z

tensor([[1.5974, 1.9384],
        [1.4729, 1.3444],
        [1.9928, 1.7016]])

In [10]:
# Add 1 and update z tensor in-place
z.add_(1)

tensor([[2.5974, 2.9384],
        [2.4729, 2.3444],
        [2.9928, 2.7016]])

In [11]:
# z has been updated
z

tensor([[2.5974, 2.9384],
        [2.4729, 2.3444],
        [2.9928, 2.7016]])

### Reshaping

Reshaping tensors is a really common operation. First to get the size and shape of a tensor use `.size()`. Then, to reshape a tensor, use `.resize_()`. Notice the underscore, reshaping is an in-place operation.

In [12]:
z.size()

torch.Size([3, 2])

In [13]:
z.resize_(2, 3)

tensor([[2.5974, 2.9384, 2.4729],
        [2.3444, 2.9928, 2.7016]])

In [14]:
z

tensor([[2.5974, 2.9384, 2.4729],
        [2.3444, 2.9928, 2.7016]])

## Numpy to Torch and back

Converting between Numpy arrays and Torch tensors is super simple and useful. To create a tensor from a Numpy array, use `torch.from_numpy()`. To convert a tensor to a Numpy array, use the `.numpy()` method.

In [15]:
a = np.random.rand(4,3)
a

array([[4.94403698e-01, 2.07303060e-01, 8.07505053e-01],
       [9.24968593e-01, 6.76515359e-01, 5.04840684e-01],
       [7.15712644e-01, 6.61123869e-01, 2.97189434e-01],
       [5.92227250e-01, 1.35619730e-05, 6.31668767e-01]])

In [16]:
b = torch.from_numpy(a)
b

tensor([[4.9440e-01, 2.0730e-01, 8.0751e-01],
        [9.2497e-01, 6.7652e-01, 5.0484e-01],
        [7.1571e-01, 6.6112e-01, 2.9719e-01],
        [5.9223e-01, 1.3562e-05, 6.3167e-01]], dtype=torch.float64)

In [17]:
b.numpy()

array([[4.94403698e-01, 2.07303060e-01, 8.07505053e-01],
       [9.24968593e-01, 6.76515359e-01, 5.04840684e-01],
       [7.15712644e-01, 6.61123869e-01, 2.97189434e-01],
       [5.92227250e-01, 1.35619730e-05, 6.31668767e-01]])

The memory is shared between the Numpy array and Torch tensor, so if you change the values in-place of one object, the other will change as well.

In [18]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)

tensor([[9.8881e-01, 4.1461e-01, 1.6150e+00],
        [1.8499e+00, 1.3530e+00, 1.0097e+00],
        [1.4314e+00, 1.3222e+00, 5.9438e-01],
        [1.1845e+00, 2.7124e-05, 1.2633e+00]], dtype=torch.float64)

In [19]:
# Numpy array matches new values from Tensor
a

array([[9.88807395e-01, 4.14606120e-01, 1.61501011e+00],
       [1.84993719e+00, 1.35303072e+00, 1.00968137e+00],
       [1.43142529e+00, 1.32224774e+00, 5.94378868e-01],
       [1.18445450e+00, 2.71239461e-05, 1.26333753e+00]])